In [1]:
from google.cloud import storage
import pandas as pd
import io
from pyspark.sql import SparkSession

In [2]:
# spark = SparkSession.builder \
#     .appName("MyPySparkJob") \
#     .config("spark.driver.memory", "32g") \
#     .config("spark.executor.memory", "32g") \
#     .config("spark.executor.cores", "4") \
#     .getOrCreate()

In [3]:

# spark = SparkSession.builder \
#   .appName('Jupyter BigQuery Storage')\
#   .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
#   .getOrCreate()

spark = SparkSession.builder \
    .appName('my-spark-job') \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.22.1') \
    .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true') \
    .config('spark.sql.catalog.spark_bigquery', 'com.google.cloud.spark.bigquery.v2.BigQueryCatalog') \
    .config('spark.sql.catalog.spark_bigquery.projectId', 'arched-glass-384816') \
    .getOrCreate()

In [4]:
spark

In [6]:
df = spark.read.format("csv") \
      .option("header", "true") \
      .load("gs://sammy-project-bucket/out.csv")

In [7]:
df.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|preferred_foot|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|          Left|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|         Right|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|


In [8]:
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [9]:
df.dropna()
df.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|preferred_foot|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|          Left|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|         Right|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|


In [10]:
df2 = df.where(df.player_positions!='GK')
df2.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|preferred_foot|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+--------------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|          Left|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|         Right|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|


In [11]:
df3 = df2.drop(df2.preferred_foot)
df3.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|
|   R. Lewandowski|              ST|     91|       91| 84000000.0|420000.0|

In [12]:
from pyspark.sql.functions import concat, col, lit
df3 = df3.withColumn('fixed_pos', concat(df3.player_positions.substr(1, 3)))
df3.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|       RW|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|      CF,|
|   R. Lewandowski|      

In [13]:
df3.select('fixed_pos').distinct().collect()

[Row(fixed_pos='RW,'),
 Row(fixed_pos='LWB'),
 Row(fixed_pos='CB,'),
 Row(fixed_pos='LM'),
 Row(fixed_pos='CAM'),
 Row(fixed_pos='LB'),
 Row(fixed_pos='LW,'),
 Row(fixed_pos='CM,'),
 Row(fixed_pos='LW'),
 Row(fixed_pos='LM,'),
 Row(fixed_pos='RB'),
 Row(fixed_pos='RB,'),
 Row(fixed_pos='CM'),
 Row(fixed_pos='RW'),
 Row(fixed_pos='CF,'),
 Row(fixed_pos='CDM'),
 Row(fixed_pos='CB'),
 Row(fixed_pos='LB,'),
 Row(fixed_pos='ST,'),
 Row(fixed_pos='RWB'),
 Row(fixed_pos='CF'),
 Row(fixed_pos='RM'),
 Row(fixed_pos='RM,'),
 Row(fixed_pos='ST')]

In [14]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
df3 = df3.withColumn('fixed_pos', 
               when(col('fixed_pos') == 'CB,', 'CB')
               .when(col('fixed_pos') == 'LB,', 'LB')
               .when(col('fixed_pos') == 'RB,', 'RB')
               .when(col('fixed_pos') == 'CM,', 'CM')
               .when(col('fixed_pos') == 'LM,', 'LM')
               .when(col('fixed_pos') == 'RM,', 'RM')
               .when(col('fixed_pos') == 'LW,', 'LW')
               .when(col('fixed_pos') == 'RW,', 'RW')
               .when(col('fixed_pos') == 'CF,', 'CF')
               .when(col('fixed_pos') == 'ST,', 'ST')
               .otherwise(col('fixed_pos'))
              )
df3.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|       RW|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|       CF|
|   R. Lewandowski|      

In [15]:
df3.select('fixed_pos').distinct().collect()

[Row(fixed_pos='LWB'),
 Row(fixed_pos='LM'),
 Row(fixed_pos='CAM'),
 Row(fixed_pos='LB'),
 Row(fixed_pos='LW'),
 Row(fixed_pos='RB'),
 Row(fixed_pos='CM'),
 Row(fixed_pos='RW'),
 Row(fixed_pos='CDM'),
 Row(fixed_pos='CB'),
 Row(fixed_pos='RWB'),
 Row(fixed_pos='CF'),
 Row(fixed_pos='RM'),
 Row(fixed_pos='ST')]

In [16]:
df3 =  df3.withColumn(
    "fixed_pos",
    when(col("fixed_pos").isin('CB', 'LB', 'RB', 'RWB', 'LWB'),"DEF")
    .when(col("fixed_pos").isin("CM", "CAM", "RM", "LM", "CDM"),"MID")
    .when(col("fixed_pos").isin("ST", "CF", "LW", "RW"),"FWD")
    .otherwise(col("fixed_pos"))
)
df3.show()

+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|       short_name|player_positions|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|
+-----------------+----------------+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|         L. Messi|              RW|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|      FWD|
|       K. Benzema|          CF, ST|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|      FWD|
|   R. Lewandowski|      

In [17]:
df3 = df3.drop(df3.player_positions)
df3 = df3.drop(df3.short_name)
df3.show()

+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|    work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|
+-------+---------+-----------+--------+---+---------+---------+---------+-----------+-------------+----+--------+-------+---------+---------+------+---------+
|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|      Low/Low|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|      FWD|
|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|Medium/Medium|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|      FWD|
|     91|       91| 84000000.0|420000.0| 33|      185|       81|        4|          4|  High/Medium|75.0|    91.0|   79.0|     86.0|     44.0|  83.0|      FWD|
|     91|       91|107500000.0|350000.0|

In [18]:
df3 = df3.withColumn('work_rate', 
               when(col('work_rate') == 'Low/Low', 1)
               .when(col('work_rate') == 'Low/Medium', 2)
               .when(col('work_rate') == 'Medium/Low', 2)
               .when(col('work_rate') == 'Medium/Medium', 3)
               .when(col('work_rate') == 'Low/High', 3)
               .when(col('work_rate') == 'High/Low', 3)
               .when(col('work_rate') == 'Medium/High', 4)
               .when(col('work_rate') == 'High/Medium', 4)
               .when(col('work_rate') == 'High/High', 5)
               .otherwise(col('work_rate'))
              )
df3.show()

+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+---------+
|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|
+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+---------+
|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|        1|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|      FWD|
|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|        3|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|      FWD|
|     91|       91| 84000000.0|420000.0| 33|      185|       81|        4|          4|        4|75.0|    91.0|   79.0|     86.0|     44.0|  83.0|      FWD|
|     91|       91|107500000.0|350000.0| 31|      181|       75|

In [19]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="fixed_pos", outputCol="fixed_pos_dummies")
indexed = indexer.fit(df3).transform(df3)
indexed.show()

+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+---------+-----------------+
|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos|fixed_pos_dummies|
+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+---------+-----------------+
|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|        1|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|      FWD|              2.0|
|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|        3|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|      FWD|              2.0|
|     91|       91| 84000000.0|420000.0| 33|      185|       81|        4|          4|        4|75.0|    91.0|   79.0|     86.0|  

In [20]:
df4 = indexed.drop(indexed.fixed_pos)
df4.show()

+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos_dummies|
+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|        1|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|              2.0|
|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|        3|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|              2.0|
|     91|       91| 84000000.0|420000.0| 33|      185|       81|        4|          4|        4|75.0|    91.0|   79.0|     86.0|     44.0|  83.0|              2.0|
|     91|       

In [21]:
df4.na.drop().show()

+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|overall|potential|  value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos_dummies|
+-------+---------+-----------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|     91|       91| 54000000.0|195000.0| 35|      169|       67|        4|          4|        1|81.0|    89.0|   90.0|     94.0|     34.0|  64.0|              2.0|
|     91|       91| 64000000.0|450000.0| 34|      185|       81|        4|          4|        3|80.0|    88.0|   83.0|     87.0|     39.0|  78.0|              2.0|
|     91|       91| 84000000.0|420000.0| 33|      185|       81|        4|          4|        4|75.0|    91.0|   79.0|     86.0|     44.0|  83.0|              2.0|
|     91|       

In [22]:
from pyspark.sql.types import IntegerType
df4 = df4.withColumn("work_rate", df4["work_rate"].cast(IntegerType()))
df4 = df4.withColumn("overall", df4["overall"].cast(IntegerType()))
df4 = df4.withColumn("value_eur", df4["value_eur"].cast(IntegerType()))
df4 = df4.withColumn("wage_eur", df4["wage_eur"].cast(IntegerType()))
df4 = df4.withColumn("age", df4["age"].cast(IntegerType()))
df4 = df4.withColumn("height_cm", df4["height_cm"].cast(IntegerType()))
df4 = df4.withColumn("weak_foot", df4["weak_foot"].cast(IntegerType()))
df4 = df4.withColumn("skill_moves", df4["skill_moves"].cast(IntegerType()))
df4 = df4.withColumn("pace", df4["pace"].cast(IntegerType()))
df4 = df4.withColumn("shooting", df4["shooting"].cast(IntegerType()))
df4 = df4.withColumn("passing", df4["passing"].cast(IntegerType()))
df4 = df4.withColumn("dribbling", df4["dribbling"].cast(IntegerType()))
df4 = df4.withColumn("defending", df4["defending"].cast(IntegerType()))
df4 = df4.withColumn("physic", df4["physic"].cast(IntegerType()))
df4 = df4.withColumn("weight_kg", df4["weight_kg"].cast(IntegerType()))
df4 = df4.withColumn("potential", df4["potential"].cast(IntegerType()))

In [23]:
df4.printSchema()

root
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- wage_eur: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullable = true)
 |-- work_rate: integer (nullable = true)
 |-- pace: integer (nullable = true)
 |-- shooting: integer (nullable = true)
 |-- passing: integer (nullable = true)
 |-- dribbling: integer (nullable = true)
 |-- defending: integer (nullable = true)
 |-- physic: integer (nullable = true)
 |-- fixed_pos_dummies: double (nullable = false)



In [24]:
df4.show()

+-------+---------+---------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|overall|potential|value_eur|wage_eur|age|height_cm|weight_kg|weak_foot|skill_moves|work_rate|pace|shooting|passing|dribbling|defending|physic|fixed_pos_dummies|
+-------+---------+---------+--------+---+---------+---------+---------+-----------+---------+----+--------+-------+---------+---------+------+-----------------+
|     91|       91| 54000000|  195000| 35|      169|       67|        4|          4|        1|  81|      89|     90|       94|       34|    64|              2.0|
|     91|       91| 64000000|  450000| 34|      185|       81|        4|          4|        3|  80|      88|     83|       87|       39|    78|              2.0|
|     91|       91| 84000000|  420000| 33|      185|       81|        4|          4|        4|  75|      91|     79|       86|       44|    83|              2.0|
|     91|       91|107500000

In [25]:
#apply ml algos

In [26]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [27]:
from pyspark.sql.types import DoubleType
df4 = df4.withColumn("work_rate", df4["work_rate"].cast(DoubleType()))
df4 = df4.withColumn("overall", df4["overall"].cast(DoubleType()))
df4 = df4.withColumn("value_eur", df4["value_eur"].cast(DoubleType()))
df4 = df4.withColumn("wage_eur", df4["wage_eur"].cast(DoubleType()))
df4 = df4.withColumn("age", df4["age"].cast(DoubleType()))
df4 = df4.withColumn("height_cm", df4["height_cm"].cast(DoubleType()))
df4 = df4.withColumn("weak_foot", df4["weak_foot"].cast(DoubleType()))
df4 = df4.withColumn("skill_moves", df4["skill_moves"].cast(DoubleType()))
df4 = df4.withColumn("pace", df4["pace"].cast(DoubleType()))
df4 = df4.withColumn("shooting", df4["shooting"].cast(DoubleType()))
df4 = df4.withColumn("passing", df4["passing"].cast(DoubleType()))
df4 = df4.withColumn("dribbling", df4["dribbling"].cast(DoubleType()))
df4 = df4.withColumn("defending", df4["defending"].cast(DoubleType()))
df4 = df4.withColumn("physic", df4["physic"].cast(DoubleType()))
df4 = df4.withColumn("weight_kg", df4["weight_kg"].cast(DoubleType()))
df4 = df4.withColumn("potential", df4["potential"].cast(DoubleType()))
df4 = df4.withColumn("value_eur", df4["value_eur"].cast(DoubleType()))

In [28]:
df4 = df4.dropna()
 
assembler = VectorAssembler(inputCols=['overall',
 'potential',
 'wage_eur',
 'age',
 'height_cm',
 'weight_kg',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic'], outputCol="features")
output = assembler.transform(df4)
final_data = output.select("features", "value_eur")

In [29]:
final_data.cache()
final_data

DataFrame[features: vector, value_eur: double]

In [30]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [31]:
from pyspark.ml.regression import LinearRegression
 
lr = LinearRegression(featuresCol="features", labelCol="value_eur")
lr_model = lr.fit(train_data)


In [32]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [230536.99345163055,14338.931311158905,285.4203276911139,-274390.60367794376,-11426.70697307964,-9319.194394868846,91657.44555559574,129050.06011552145,149970.6468219952,18171.299259095147,8062.305825221805,19106.855663257218,-57480.89123487609,-8206.609804405301,16528.823888660074]
Intercept: -7244409.819764829


In [33]:
#Summarize the model over the training set and print out some metrics:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("R squre: %f" % trainingSummary.r2)

RMSE: 4303258.218086
R squre: 0.700818


In [34]:
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("prediction","value_eur","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="value_eur",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+---------+--------------------+
|         prediction|value_eur|            features|
+-------------------+---------+--------------------+
|-3388463.0903736954| 100000.0|[45.0,53.0,550.0,...|
|-3388463.0903736954| 100000.0|[45.0,53.0,550.0,...|
|-3388463.0903736954| 100000.0|[45.0,53.0,550.0,...|
|-2525725.9334944403| 100000.0|[45.0,56.0,650.0,...|
|-2525725.9334944403| 100000.0|[45.0,56.0,650.0,...|
+-------------------+---------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.702511


In [35]:
test_result = lr_model.evaluate(test_data)
print("Root Mean Squared Error (RMSE) on test data = %f" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 4254103.053684


In [51]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="features", labelCol="value_eur", numTrees=10, maxDepth=5, seed=42)
model = rf.fit(train_data)

In [52]:
predictions = model.transform(test_data)

In [54]:
evaluator = RegressionEvaluator(metricName="rmse", predictionCol="prediction", labelCol="value_eur")
rmse = evaluator.evaluate(predictions)
print("RMSE on test data = %g" % rmse)

RMSE on test data = 1.54388e+06


In [36]:
# from pyspark.sql.functions import rand
 
# # Split the dataset into training and test sets
# train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)
 
# # Select X and y for the training set
# train_X = train_df.select(
#     *[x for x in train_df.columns if x not in ["value_eur"]]
# )
# train_y = train_df.select("value_eur")
 
# # Select X and y for the test set
# test_X = test_df.select(
#     *[x for x in test_df.columns if x not in ["value_eur"]]
# )
# test_y = test_df.select("value_eur")

In [37]:
#Check shape of train and test dataset
# print("shape of train_X: ", (train_X.count(), len(train_X.columns)))
# print("shape of train_y: ", (train_y.count(), len(train_y.columns)))
# print("shape of test_X: ", (test_X.count(), len(test_X.columns)))
# print("shape of test_y: ", (test_y.count(), len(test_y.columns)))

In [38]:
# train_df.columns

In [39]:
# from pyspark.ml.feature import VectorAssembler
# vectorAssembler = VectorAssembler(inputCols = ['overall',
#  'potential',
#  'wage_eur',
#  'age',
#  'height_cm',
#  'weight_kg',
#  'weak_foot',
#  'skill_moves',
#  'work_rate',
#  'pace',
#  'shooting',
#  'passing',
#  'dribbling',
#  'defending',
#  'physic'], outputCol = 'features')
# df = vectorAssembler.transform(df4)
# df = df.select(['features', 'value_eur'])
# df.show(3)

In [40]:
# df.withColumnRenamed("value_eur","label").show()

In [41]:

# (trainingData, testData) = df.randomSplit([0.7, 0.3])

In [42]:
# trainingData


In [43]:
# from pyspark.ml import Pipeline
# from pyspark.ml.regression import RandomForestRegressor
# from pyspark.ml.feature import VectorIndexer
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.classification import DecisionTreeClassifier
# # Train a RandomForest model.

# #tree = DecisionTreeClassifier(featuresCol='features', labelCol='value_eur')

# rf = RandomForestRegressor(featuresCol='features', labelCol='value_eur')
# model = rf.fit(trainingData)

In [44]:
# splits = df.randomSplit([0.7, 0.3], seed=42)
# train_df = splits[0]
# test_df = splits[1]

In [45]:
# train_df.show()

In [46]:
# # Define the linear regression model
# from pyspark.ml.regression import LinearRegression

# lr = LinearRegression(featuresCol = 'features', labelCol='value_eur', maxIter=10, regParam=0.3, elasticNetParam=0.8)
# lr_model = lr.fit(train_df)
# print("Coefficients: " + str(lr_model.coefficients))
# print("Intercept: " + str(lr_model.intercept))

In [47]:
# from xgboost.spark import SparkXGBRegressor
# spark_reg_estimator = SparkXGBRegressor(
#   features_col="features",
#   label_col="value_eur",
#   num_workers=3,
# )

In [48]:
# from pyspark.ml.regression import LinearRegression
# from pyspark.ml import Pipeline
# from pyspark.ml.regression import RandomForestRegressor

In [49]:
# train_df.show()